In [1]:
from algosdk import account, mnemonic

private_key, address = account.generate_account()
print("My address: {}".format(address))
print("My private key: {}".format(private_key))
print("My passphrase: {}".format(mnemonic.from_private_key(private_key)))


My address: F3ONYOMHSPO5DKF2Z744DBB5ZQX5LMJAKMUWLHAD43LEU4OPWGF5QAFHGA
My private key: AogEoPWnBiqPBsE/KzqwHBXwjmWY8aBog5YJRSdMykQu3Nw5h5Pd0ai6z/nBhD3ML9WxIFMpZZwD5tZKcc+xiw==
My passphrase: able banana source leg ask skin crucial scissors gun trim fix fade then glove segment various beef asset slender explain excess narrow chaos able game


In [2]:
mnemo2 = 'tribe leopard seek wage genre affair section topic pulse end rare enough butter hope buzz arctic night truly muffin breeze repeat limit child ability turtle'
priv_2 = mnemonic.to_private_key(mnemo2)
pub_2 = mnemonic.to_public_key(mnemo2)

In [3]:
priv_2

'QhcghmOPMBFUGOVsZRJkr6SPtfEjC6xceiK5UdsH/iReuSoQnLCrSqTm3JGZlcFcHgEdcfcEibrBs3DBCdYgOA=='

In [5]:
from algosdk.v2client import algod

algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"
algod_client = algod.AlgodClient(algod_token, algod_address)

In [6]:
account_info = algod_client.account_info(pub_2)
print("Account balance: {} microAlgos".format(account_info.get('amount')) + "\n")


Account balance: 4000808000000000 microAlgos



In [9]:
# build transaction
from algosdk.future import transaction
from algosdk import constants


params = algod_client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
params.flat_fee = True
params.fee = constants.MIN_TXN_FEE 
receiver = address
note = "Hello World".encode()
amount = 1000000
unsigned_txn = transaction.PaymentTxn(pub_2, params, receiver, amount, None, note)


In [11]:
# sign transaction
signed_txn = unsigned_txn.sign(priv_2)


In [13]:
txid = algod_client.send_transaction(signed_txn)

In [14]:
import json
import base64


    #submit transaction
    
print("Successfully sent transaction with txID: {}".format(txid))

    # wait for confirmation 
try:
    confirmed_txn = transaction.wait_for_confirmation(algod_client, txid, 4)  
except Exception as err:
    print(err)
    #return

print("Transaction information: {}".format(
    json.dumps(confirmed_txn, indent=4)))
print("Decoded note: {}".format(base64.b64decode(
    confirmed_txn["txn"]["txn"]["note"]).decode()))
print("Starting Account balance: {} microAlgos".format(account_info.get('amount')) )
print("Amount transfered: {} microAlgos".format(amount) )    
print("Fee: {} microAlgos".format(params.fee) ) 


account_info = algod_client.account_info(pub_2)
print("Final Account balance: {} microAlgos".format(account_info.get('amount')) + "\n")


Successfully sent transaction with txID: ZTGYQWW7QS6UG6BQRJ4C6XHPGAME6KIADZNUJLA7DS7RKVU7JUQQ
Transaction information: {
    "confirmed-round": 8850,
    "pool-error": "",
    "sender-rewards": 884000000000,
    "txn": {
        "sig": "9RNTER+B8QD/owXOBHGWx4M2TyWfYgfx1QWsMyrju7p1eoanY30YCY9mn2/z50vmHJbIl+Vr9PXBjztFbsX8Ag==",
        "txn": {
            "amt": 1000000,
            "fee": 1000,
            "fv": 8160,
            "gen": "sandnet-v1",
            "gh": "vvErYnu7ztsIY+DuOBVCeTFM3JSJk5JbSXLOGE6KVc8=",
            "lv": 9160,
            "note": "SGVsbG8gV29ybGQ=",
            "rcv": "F3ONYOMHSPO5DKF2Z744DBB5ZQX5LMJAKMUWLHAD43LEU4OPWGF5QAFHGA",
            "snd": "L24SUEE4WCVUVJHG3SIZTFOBLQPACHLR64CITOWBWNYMCCOWEA4G4KOUYU",
            "type": "pay"
        }
    }
}
Decoded note: Hello World
Starting Account balance: 0 microAlgos
Amount transfered: 1000000 microAlgos
Fee: 1000 microAlgos
Final Account balance: 4000887999882998 microAlgos



In [15]:
account_info = algod_client.account_info(address)
print("Final Account balance: {} microAlgos".format(account_info.get('amount')) + "\n")

Final Account balance: 1000001 microAlgos



# ASSETS

In [16]:
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn

# CREATE ASSET
# Get network params for transactions before every transaction.
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
# params.fee = 1000
# params.flat_fee = True
# Account 1 creates an asset called latinum and
# sets Account 2 as the manager, reserve, freeze, and clawback address.
# Asset Creation transaction
txn = AssetConfigTxn(
    sender=pub_2,
    sp=params,
    total=1000,
    default_frozen=False,
    unit_name="LATINUM",
    asset_name="latinum",
    manager=pub_2,
    reserve=pub_2,
    freeze=pub_2,
    clawback=pub_2,
    url="https://path/to/my/asset/details", 
    decimals=0)
# Sign with secret key of creator
stxn = txn.sign(priv_2)
# Send the transaction to the network and retrieve the txid.
try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4)  
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))   
except Exception as err:
    print(err)
# Retrieve the asset ID of the newly created asset by first
# ensuring that the creation transaction was confirmed,
# then grabbing the asset id from the transaction.
print("Transaction information: {}".format(
    json.dumps(confirmed_txn, indent=4)))
# print("Decoded note: {}".format(base64.b64decode(
#     confirmed_txn["txn"]["txn"]["note"]).decode()))



Signed transaction with txID: T7KJXGUDKYWGIHO2AMBKBBHCKSADREPLS5S7RUTO33JSRHDEUOXA
name 'wait_for_confirmation' is not defined
Transaction information: {
    "confirmed-round": 8850,
    "pool-error": "",
    "sender-rewards": 884000000000,
    "txn": {
        "sig": "9RNTER+B8QD/owXOBHGWx4M2TyWfYgfx1QWsMyrju7p1eoanY30YCY9mn2/z50vmHJbIl+Vr9PXBjztFbsX8Ag==",
        "txn": {
            "amt": 1000000,
            "fee": 1000,
            "fv": 8160,
            "gen": "sandnet-v1",
            "gh": "vvErYnu7ztsIY+DuOBVCeTFM3JSJk5JbSXLOGE6KVc8=",
            "lv": 9160,
            "note": "SGVsbG8gV29ybGQ=",
            "rcv": "F3ONYOMHSPO5DKF2Z744DBB5ZQX5LMJAKMUWLHAD43LEU4OPWGF5QAFHGA",
            "snd": "L24SUEE4WCVUVJHG3SIZTFOBLQPACHLR64CITOWBWNYMCCOWEA4G4KOUYU",
            "type": "pay"
        }
    }
}
